In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import decomposition, preprocessing, linear_model, discriminant_analysis, neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, recall_score, roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import scale
import itertools
# from statMlFunctions import *
#import seaborn as sns
#sns.set(context='paper', style='whitegrid', color_codes=True, font_scale=1.8)
#sns.set_palette(sns.color_palette("Set1", n_colors=12, desat=.5))
import seaborn as sns
sns.set(context='paper', style='whitegrid', color_codes=True, font_scale=1.8)
colorcycle = [(0.498, 0.788, 0.498),
              (0.745, 0.682, 0.831),
              (0.992, 0.753, 0.525),
              (0.220, 0.424, 0.690),
              (0.749, 0.357, 0.090),
              (1.000, 1.000, 0.600),
              (0.941, 0.008, 0.498),
              (0.400, 0.400, 0.400)]
sns.set_palette(colorcycle) 

%matplotlib inline
# sns.set(style="white")
# mpl.rcParams['figure.figsize'] = [10, 6]

### Load data

In [ ]:
df0 = pd.read_table('RGender.dat', sep=' ').T
df0.head() 

In [ ]:
df1 = df0.drop('gender', axis=1)
df1.rename(columns=lambda s: s.replace('bfi_', '').replace('.answer', ''), inplace=True)
df1.head() 

In [ ]:
gender = df0.gender

In [ ]:
gender.value_counts() 

In [ ]:
df1.shape

### Plot cross correlation

Remove this, since the cross correlation between the variables aren't af any real interest?

In [ ]:
mpl.rcParams['text.usetex'] = False
mpl.rcParams['text.latex.unicode'] = False

def plotCrossCorr(df, threshold=None):
    
    df = df.copy() 
    corr = df.corr()
    corrval = corr.values
    mask = np.eye(*corr.shape)
    if threshold is None:
        mask[np.triu_indices_from(mask)] = 1
    else:
        mask[np.triu_indices_from(mask, k=1)] = np.abs(corrval[np.triu_indices_from(mask, k=1)]) < threshold
    fig, ax = plt.subplots(figsize=(12, 11))
    cmap = sns.diverging_palette(10, 220, as_cmap=True)
    sns.heatmap(corr, vmin=-1, vmax=1, cmap=cmap, mask=mask, linewidths=0.8, ax=ax)
    return fig, ax, corrval

fig, ax, corrval = plotCrossCorr(df1, threshold=0.5);
fig.savefig('data_cross_correlation.pdf')


class LargestElements:
    """Container for n absolute largest elements"""
    def __init__(self, n):
        self.n = n
        self.absdct = dict()

    def add(self, val, i, j):
        absval = np.abs(val)
        keys = self.absdct.keys()
        if len(self.absdct) < self.n or absval > min(keys):
            self.absdct[absval] = (val, i, j)
        if len(self.absdct) > self.n:
            del self.absdct[min(keys)]

    def get(self):
        return self.absdct


def max_abs_n_idx_triu(arr, n):
    le = LargestElements(n)
    for i in range(arr.shape[0]):
        for j in range(i):
            if i == j:
                continue
            le.add(arr[i, j], i, j)
    return le.get()


max_abs_n_idx_triu(corrval, 4)

## Logistic regression on the raw data

In [ ]:
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.unicode'] = True

In [ ]:
lr = linear_model.LogisticRegression(penalty='l2')
lr.fit(df1.values, gender.values)
prob1 = lr.predict_proba(df1.values)[:, 1]

In [ ]:
accuracy_score(gender.values, prob1>0.5)

In [ ]:
confusion_matrix(gender.values, prob1>0.5)

In [ ]:
len([(t, p) for (t, p) in zip(gender.values.astype(int), (prob1>0.5).astype(int)) if (t & p)])

In [ ]:
len([(t, p) for (t, p) in zip(gender.values.astype(int), (prob1>0.5).astype(int)) if ((t==0) & (p==0))])

In [ ]:
fig, ax = plt.subplots()
ax.hist(prob1, bins=50)
ax.set_xlabel('$p_f$')
ax.set_ylabel('Count')
fig.savefig('sex_prediction_raw_data_prob1_vs_count.pdf')

In [ ]:
fpr, tpr, threshold = roc_curve(gender.values, prob1)
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=4) 
ax.set_ylabel('TPR')
ax.set_xlabel('FPR')
fig.savefig('sex_prediction_raw_data.pdf')
print("AUC:", auc(fpr, tpr)) 

### Logistic regression on the _expanded_ raw data

Expand the data with products accross _all_ variables

In [ ]:
idx = np.arange(df1.values.shape[1]).astype(int)
idxCombinations = list(itertools.product(idx, idx))
featureMatrix = np.NaN * np.zeros((df1.values.shape[0], df1.values.shape[1]+len(idxCombinations)))

In [ ]:
featureMatrix[:, :df1.values.shape[1]] = df1.values.copy() 

In [ ]:
for i, (v0, v1) in enumerate(idxCombinations, df1.values.shape[1]):
    featureMatrix[:, i] = featureMatrix[:, v0] * featureMatrix[:, v1]

In [ ]:
lre = linear_model.LogisticRegression()
lre.fit(featureMatrix, gender.values)
prob1 = lre.predict_proba(featureMatrix)[:, 1]

In [ ]:
fig, ax = plt.subplots()
ax.hist(prob1, bins=50)
ax.set_xlabel('$p_f$')
ax.set_ylabel('Count')
fig.savefig('sex_prediction_expanded_data_prob1_vs_count.pdf')

In [ ]:
accuracy_score(gender.values, prob1>0.5) 

In [ ]:
fpr, tpr, threshold = roc_curve(gender.values, prob1)
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=4) 
ax.set_ylabel('TPR')
ax.set_xlabel('FPR')
fig.savefig('sex_prediction_expanded_data.pdf')
print(f"AUC: {auc(fpr, tpr)}", f"FPR: {fpr}", f"TPR: {tpr}", sep='\n') 

### Perform PCA 

In [ ]:
nd1 = scale(df1.values)
pca = decomposition.PCA(svd_solver='full') 
pca.fit(nd1)

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 6)) 
ax0.plot(pca.explained_variance_ratio_, 'o-')
ax1.plot(np.cumsum(pca.explained_variance_ratio_), 'o-')
ax0.set_title('Eigenvalues')
ax1.set_title('Eigenvalues accumulated')

In [ ]:
# For saving output

fig, ax = plt.subplots() 
ax.plot(pca.explained_variance_ratio_, 'o-')
ax.set_xlabel('Eigenvalue #')
ax.set_ylabel('Eigenvalues')
plt.tight_layout() 
fig.subplots_adjust(left=0.15, bottom=0.15)
fig.savefig('pca_eigenvalues.pdf')

fig, ax = plt.subplots()
ax.plot(np.cumsum(pca.explained_variance_ratio_), 'o-')
ax.set_xlabel('Eigenvalue #')
ax.set_ylabel('Eigenvalues accumulated')
plt.tight_layout() 
fig.subplots_adjust(bottom=0.15)
fig.savefig('pca_eigenvalues_accumulated.pdf')

fig, ax = plt.subplots()
ax.plot(1 - np.cumsum(pca.explained_variance_ratio_), 'o-')
ax.set_xlabel('Eigenvalue #')
ax.set_ylabel('1 - Eigenvalues accumulated')
plt.tight_layout() 
fig.subplots_adjust(bottom=0.15)
fig.savefig('pca_eigenvalues_scree.pdf')

In [ ]:
fig, ax = plt.subplots()
ax.hist(pca.explained_variance_ratio_, bins=65);
ax.set_xlabel('Normalized eigenvalues')
ax.set_ylabel('Count')
fig.savefig('pca_eigenvalues_histogram.pdf')

In [ ]:
td1 = pca.transform(nd1)  # The rotated vector space

In [ ]:
td1[:, 0].std() 

In [ ]:
td1.shape

In [ ]:
td1[0, :]

In [ ]:
td1.shape

### Logistic Regression

Try the prediction with a different number of leading vectors, and plot the accuracy for the prediction

In [ ]:
res = np.zeros((9, 2))
fig, ax = plt.subplots()
for k in range(1, 10):
    lr = linear_model.LogisticRegressionCV()
    lr.fit(td1[:, :k], gender.values)
    td1p = lr.predict(td1[:, :k]) 
    acc = accuracy_score(gender.values, td1p)
    print("Accuracy using {} leading component(s) is {:.2f} % ".format(k, acc*100))
    res[k-1, 0] = k
    res[k-1, 1] = acc
ax.plot(res[:, 0], res[:, 1], '-o')
ax.set_xlabel("Number of components")
ax.set_ylabel("Accuracy");
ax.set_xticks(np.arange(1, 10))
fig.savefig('pca_linear_regression_accuracy_vs_components.pdf')

In [ ]:
res = np.zeros((9, 2))
fig, ax = plt.subplots()
for reg in 'l1 l2'.split():
    for k in range(1, 10):
        lr = linear_model.LogisticRegressionCV(penalty=reg, solver='liblinear')
        lr.fit(td1[:, :k], gender.values)
        td1p = lr.predict(td1[:, :k]) 
        acc = (td1p == gender.values).sum() / gender.size * 100
        print("Accuracy using {} leading component(s) is {:.2f} % ".format(k, acc))
        res[k-1, 0] = k
        res[k-1, 1] = acc
    ax.plot(res[:, 0], res[:, 1], '-o', label=reg.upper())
ax.set_xlabel("Number of components")
ax.set_ylabel("Accuracy");
ax.set_yticks([78, 80, 82])
ax.set_xticks(np.arange(1, 10))
ax.legend(loc="lower right")
fig.savefig('pca_linear_regression_L1_L2_accuracy_vs_components.pdf')

Fit a Logistic Regression prediction with $k = 5$ components, and plot each vector component against the gender and the prediction probability

#### Elastic net regression: Combining L1 and L2 regularization

In [ ]:
res = np.zeros((9, 2))
fig, ax = plt.subplots()
for r in np.linspace(0.1, 0.9, 9):
    for k in range(1, 10):
        lr = linear_model.ElasticNetCV(l1_ratio=r)
        lr.fit(td1[:, :k], gender.values)
        td1p = lr.predict(td1[:, :k]) 
        acc = accuracy_score(gender.values, td1p > 0.5) * 100
        # print("Accuracy using {} leading component(s) is {:.2f} % ".format(k, acc))
        res[k-1, 0] = k
        res[k-1, 1] = acc
    ax.plot(res[:, 0], res[:, 1], '-o', label='r=%.1f' % r)
ax.set_xlabel("Number of components")
ax.set_ylabel("Accuracy");
ax.set_yticks([78, 80, 82])
ax.set_xticks(np.arange(1, 10));
ax.legend() 
fig.savefig('pca_linear_regression_elastic_net_accuracy_vs_components.pdf')

In [ ]:
k = 5
X = td1[:, :k]  # fit using the first k components
lr = linear_model.LogisticRegression()
lr.fit(X, gender.values)
prob1 = lr.predict_proba(X)[:, 1]

In [ ]:
fig, axi = plt.subplots(2, 3, figsize=(18, 7))
axi = axi.flatten()
for i in range(k):
    axi[i].scatter(X[:, i], prob1, s=5)
    axi[i].scatter(X[:, i], gender.values, s=12, alpha=0.35)
    axi[i].set_title("Component %d" % i)
axi[-1].remove() 

Make the plots again, but save in indivudial files

In [ ]:
fig, ax = plt.subplots()
for i in range(k):
    ax.cla() 
    ax.scatter(X[:, i], prob1, s=5)
    ax.scatter(X[:, i], gender.values, s=12, alpha=0.35)
    fig.savefig(f'pca_component_{i}_vs_prob1.pdf')


Component 3 looks interesting. Check the accuracy with just that component.

More thorough examination of the components

In [ ]:
fig, axi = plt.subplots(5, 5, figsize=(18, 7))
axi = axi.flatten()
ac = 0
aucmat = np.zeros((k, k))
lr2 = linear_model.LogisticRegression(solver='newton-cg')
for i in range(k):
    X = td1[:, i][:, np.newaxis]
    lr2.fit(X, gender.values)
    for j in range(k):
        XX = td1[:, j]
        prob1 = lr2.predict_proba(XX[:, np.newaxis])[:, 1]
        axi[ac].scatter(XX, prob1, s=5)
        axi[ac].scatter(XX, gender.values, s=12, alpha=0.35)
        # axi[ac].set_title(f"fit: X[:,{i}], predict: X[:,{j}]")
        ac += 1
        aucmat[i, j] = roc_auc_score(gender.values, prob1)
fig.set
fig.savefig('gridplot.png', dpi=400)
print(aucmat)

### ROC curves

In [ ]:
fpr, tpr, threshold = roc_curve(gender.values, lr.predict_proba(td1[:, :k])[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=4) 
print("AUC:", auc(fpr, tpr))
fig.savefig('logistic_regression_roc_curve.pdf')

### Randomly sampled components

Randomly sample $k = 5$ components 1000 times, collect the AUC-scores and plot a histogram over them.

In [ ]:
k = 5
N = 5000
idx_set = set()
while len(idx_set) < N:
    idx_set.add(tuple(list(set(np.random.choice(np.arange(df1.shape[1], dtype=int), k*3)))[:k]))

lr = linear_model.LogisticRegression()
auc_scores = np.zeros(N)
for i, idx in enumerate(idx_set):
    # idx = list(set(np.random.choice(np.arange(df1.shape[1], dtype=int), k*3)))[:k] # k unique indexes
    dfX = df1.iloc[:, idx]
    X = dfX.values
    lr.fit(X, gender.values)
    auc_scores[i] = roc_auc_score(gender.values, lr.predict_proba(X)[:, 1])
fig, ax = plt.subplots()
ax.hist(auc_scores, bins=75);
ax.set_xlabel('AUC scores')
ax.set_ylabel('Count')
fig.savefig(f'logistic_regression_auc_score_{k}_random_feature_sampling.pdf')

## Split data for cross validation


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(td1, gender.values, test_size=0.3, stratify=gender.values)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### Use Linear Discriminant analysis

In [ ]:
lda = discriminant_analysis.LinearDiscriminantAnalysis(store_covariance=True)

In [ ]:
lda.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, lda.predict(X_test))

LDA didn't fare that good, but it's comparable to the SVM accuracy.

#### Also try QDA

It fares even worse, but a grid search is in order!

In [ ]:
qda = discriminant_analysis.QuadraticDiscriminantAnalysis(store_covariances=True) 

In [ ]:
qda.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, qda.predict(X_test))

### Use kNN

A real grid search with cross validation should be performed.

And ROC-curves shoud be created.

In [ ]:
knn = neighbors.KNeighborsClassifier(p=4, n_neighbors=12)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, knn.predict(X_test))

### Use some support vector machines

Train a support vector machines

In [ ]:
clf = SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X_train, y_train, cv=5)
scores

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Not a bad accuracy, but let's do a real grid search for parameter optimization.

### Do a real cross validation grid search?

Do this: `file:///Users/allan/Library/Application%20Support/Dash/User%20Contributed/Scikitlearn/Scikit-learn.docset/Contents/Resources/Documents/modules/cross_validation.html#the-cross-validate-function-and-multiple-metric-evaluation`

Supress warnings about invalied measure due to category with 0 results

In [ ]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split


In [ ]:
tuned_parameters = [{'C': [0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']}]
scores = ['precision_macro', 'recall_macro', 'accuracy']
print(tuned_parameters, scores)

In [ ]:
for score in scores:
    clf = GridSearchCV(linear_model.LogisticRegression(), param_grid=tuned_parameters,
                       scoring=score, n_jobs=32) 
    clf.fit(X_train, y_train)
    print(f"""----------------------------------------------------------------
    Best parameters set found on development set using {score} for evaluation:

    {clf.best_params_}

    Grid scores on development set:
    """)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    
    print("""

    Detailed classification report:

    The model is trained on the full development set.
    The scores are computed on the full evaluation set.
    """)

    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


In [ ]:
lr = linear_model.LogisticRegression(**clf.best_params_)
lr.fit(X_train, y_train) 

In [ ]:
fpr, tpr, threshold = roc_curve(gender.values, lr.predict_proba(td1)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=6) 
print("AUC:", auc(fpr, tpr)) 
fig.savefig('logistic_regression_all_data_predict_train_data_training_roc_curve.pdf')

In [ ]:
fpr, tpr, threshold = roc_curve(y_train, lr.predict_proba(X_train)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=6) 
print("AUC:", auc(fpr, tpr)) 
fig.savefig('logistic_regression_training_data_roc_curve.pdf')

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, lr.predict_proba(X_test)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=6) 
print("AUC:", auc(fpr, tpr)) 
fig.savefig('logistic_regression_test_data_roc_curve.pdf')

In [ ]:
tuned_parameters = [{'kernel': ['rbf', 'linear', 'poly'],
                     'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [0.1, 1, 10, 100, 1000]}]
scores = ['precision', 'recall', 'accuracy']
print(tuned_parameters, scores)

In [ ]:
for score in scores:
    clf = GridSearchCV(SVC(), param_grid=tuned_parameters,
                       scoring="%s_macro" % score, n_jobs=32) 
    clf.fit(X_train, y_train)
    print(f"""----------------------------------------------------------------
    Best parameters set found on development set using {score} for evaluation:

    {clf.best_params_}

    Grid scores on development set:
    """)
    
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    
    print("""

    Detailed classification report:

    The model is trained on the full development set.
    The scores are computed on the full evaluation set.
    """)

    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


In [ ]:
svc = SVC(probability=True, **clf.best_params_) 
svc.fit(X_train, y_train) 

In [ ]:
fpr, tpr, threshold = roc_curve(gender.values, svc.predict_proba(td1)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=6) 
print("AUC:", auc(fpr, tpr)) 
fig.savefig('svm_all_data_predict_train_data_training_roc_curve.pdf')

In [ ]:
fpr, tpr, threshold = roc_curve(y_train, svc.predict_proba(X_train)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=6) 
print("AUC:", auc(fpr, tpr)) 
fig.savefig('svm_training_data_roc_curve.pdf')

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, svc.predict_proba(X_test)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=6) 
print("AUC:", auc(fpr, tpr)) 
fig.savefig('svm_test_data_roc_curve.pdf')

In [ ]:
tuned_parameters = [dict(n_estimators=np.ceil(2**np.linspace(1, 8, 10)).astype(int),
                         max_depth=[3, 5, 8, 12, 16, 25])]

print(tuned_parameters)

In [ ]:
for score in scores:
    clf = GridSearchCV(RandomForestClassifier(), param_grid=tuned_parameters, scoring="%s_macro" % score, n_jobs=32) 
    clf.fit(X_train, y_train)
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()
    
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
clf.best_estimator_

In [ ]:
fpr, tpr, threshold = roc_curve(gender.values, clf.best_estimator_.predict_proba(td1)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=12)
print("AUC:", auc(fpr, tpr))
fig.savefig('random_forrest_all_data_roc_curve.pdf')

In [ ]:
fpr, tpr, threshold = roc_curve(y_train, clf.best_estimator_.predict_proba(X_train)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=12)
print("AUC:", auc(fpr, tpr))
fig.savefig('random_forrest_training_data_roc_curve.pdf')

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, clf.best_estimator_.predict_proba(X_test)[:, 1])
fig, ax = plt.subplots()
ax.plot(fpr, tpr, '.-', lw=0.5, markersize=12)
print("AUC:", auc(fpr, tpr))
fig.savefig('random_forrest_test_data_roc_curve.pdf')